In [ ]:
import pandas as pd
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
def get_vessel_type_from_full_path(path):
    res = get_vessel_type(path.split("/")[-1])
    if not res:
        res = get_vessel_type(path.split("/")[-2])
    return res

def get_vessel_type(name):
    if "cancer" in name.lower():
        return "Cancer"
    if "artery" in name.lower():
        return "Artery"
    if "vein" in name.lower():
        return "Vein"
    if "isv" in name.lower():
        return "ISV"
    return False

In [ ]:
parent = Path("/media/yngve/TOSHIBA EXT (YNGVE)/fish_data/organised/")
vessels = sorted({p.parent for p in parent.glob("**/*red ch_*.ims")})
data = []
for vessel in tqdm(vessels):
    try:
        data.append(pd.read_csv(vessel / "median_results.csv"))
        data[-1]["File"] = str(vessel).split("organised/7 DAY OLD Fish ")[1]
    except FileNotFoundError:
        print("No data for", vessel, flush=True)

In [ ]:
results = pd.concat(data)
results["With tumour"] = results["File"].map(lambda x: "with " in x)

In [ ]:
results["Vessel type"] = results["File"].map(get_vessel_type_from_full_path)

In [ ]:
plt.figure(figsize=(8, 4.5), dpi=200)
sns.boxplot(x="Vessel type", y="Pressure drop (Lee) [Pa/µm]", data=results)

In [ ]:
plt.figure(figsize=(8, 4.5), dpi=200)
sns.boxplot(x="Vessel type", y="Pressure drop (Pries) [Pa/µm]", data=results)

In [ ]:
results.groupby("Vessel type").median()[[
    'v [µm/s]',
    'Distance to centerline [µm]',
    'Max R [µm]',
    "Pressure drop (Lee) [Pa/µm]",
    "Pressure drop (Pries) [Pa/µm]",
    "Pressure drop (integral based) (Lee) [Pa/µm]",
    "Pressure drop (integral based) (Pries) [Pa/µm]"
]]